<a href="https://colab.research.google.com/github/cobusgreyling/hugging-face/blob/main/phi_2_offline_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /private/var/folders/6d/6svky4ks0bj_py5wk86hz5k40000gn/T/pip-req-build-lqw8p136
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/6d/6svky4ks0bj_py5wk86hz5k40000gn/T/pip-req-build-lqw8p136
  Resolved https://github.com/huggingface/transformers to commit f745e7d3f902601686b83c7cce2660c2a94509f0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install sentencepiece accelerate einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 29.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 27.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 18.2 MB/s eta 0:00:00


In [15]:
import torch

In [16]:
x = torch.rand(5, 3)
print(x)
torch.cuda.is_available()

tensor([[0.6143, 0.9326, 0.1963],
        [0.9102, 0.0308, 0.5327],
        [0.5640, 0.3481, 0.4785],
        [0.3779, 0.2651, 0.4443],
        [0.7524, 0.4165, 0.8198]])


False

In [5]:
from huggingface_hub import snapshot_download

In [6]:
repo_id = 'microsoft/phi-2'

model_path = snapshot_download(repo_id=repo_id,repo_type="model", local_dir="./phi-2", local_dir_use_symlinks=False)

model_path = "./phi-2"

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1212: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

LICENSE:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.80k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Needs 12GB of vRAM to run in float32 (default)
# Run this line to load in float16. You need Gb of vRAM
torch.set_default_dtype(torch.float16)

model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [9]:
model.device

device(type='mps', index=0)

In [10]:
import time
def generate(prompt: str, generation_params: dict = {"max_length":200})-> str :
    s = time.time()

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, **generation_params)
    completion = tokenizer.batch_decode(outputs)[0]

    elapsed = time.time() - s

    num_input_tokens = inputs['input_ids'].shape[1]
    num_total_tokens = outputs.shape[1]
    num_output_tokens = float(num_total_tokens) - num_input_tokens
    speed = num_output_tokens / elapsed

    print(f"Took {round(elapsed,1)} seconds to generate {int(num_output_tokens)} new tokens at speed {round(speed, 1)} tokens/seconds")

    return completion

In [11]:
prompt = """
Create a list of 5 words which have a similar meating to the word hope.

"""

In [17]:
result = generate(prompt, generation_params={"max_length":200})
result

AssertionError: Torch not compiled with CUDA enabled

In [18]:
result = generate(prompt, generation_params={"max_length":200})
result

AssertionError: Torch not compiled with CUDA enabled

In [19]:
result = generate(prompt, generation_params={"max_length":2000})
result

AssertionError: Torch not compiled with CUDA enabled